In [ ]:
pip install seaborn

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import logging
import os
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import BatchNormalization
from keras.utils import plot_model
from sklearn.metrics import confusion_matrix
import seaborn as sns
np.set_printoptions(suppress=True)

In [ ]:
# Memory fix
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    for gpu in gpus:
       tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
# Working directories
data_dir = 'C:/Users/BITMAP/python/data_rgb/'
working_dir = 'C:/Users/BITMAP/python/training'

In [ ]:
# Initiate logger
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(message)s')

In [ ]:
# Define parameters
batch_size =16
image_height = 128
image_width = 128
color_mode = 'rgb'
no_of_channels = 3
epochs = 10
#folds = 10
split_percentage = 0.25

In [ ]:
#Must change after every training
training_no = 11

In [ ]:
# Create directory for training
training_dir = os.path.join(working_dir, f'training_{training_no}')
if not os.path.exists(training_dir):
    os.makedirs(training_dir)
else:
    raise SystemExit("Training directory is already created!")

In [ ]:
# Import dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    class_names=['both', 'crackles', 'wheezes', 'normal'],
    image_size=(image_height, image_width),
    batch_size=batch_size,
    color_mode=color_mode)

AUTOTUNE = tf.data.AUTOTUNE
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
seed_train_validation = 1 # Must be same for train_ds and val_ds
shuffle_value = True
validation_split = 0.25

train_ds = tf.keras.utils.image_dataset_from_directory(
directory ='C:/Users/BITMAP/python/data_rgb/',
image_size = (128, 128),
validation_split = validation_split,
subset = "training",
seed = seed_train_validation,
color_mode = 'rgb',
shuffle = shuffle_value)

val_ds = tf.keras.utils.image_dataset_from_directory(
directory ='C:/Users/BITMAP/python/data_rgb/',
image_size = (128, 128),
validation_split = validation_split,
subset = "validation",
seed = seed_train_validation,
color_mode = 'rgb',
shuffle = shuffle_value)

determine how many batches of data are available in the validation set using tf.data.experimental.cardinality, and then move the two-third of them (2/3 of 30% = 20%) to a test set as follows

In [ ]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take((2*val_batches) // 3)
val_ds = val_ds.skip((2*val_batches) // 3)

In [ ]:
print("Train: " + str(len(train_ds)))
print("Val: " + str(len(val_ds)))
print("Test: " + str(len(test_ds)))

In [ ]:

input = keras.layers.Input(shape=(image_height, image_width, no_of_channels), name="input")
x = keras.layers.Rescaling(1./255)(input)
x = keras.layers.Conv2D(32, 3, strides=(1, 3), padding='same')(x)
x = keras.layers.LeakyReLU(alpha=0.1)(x)
x = keras.layers.MaxPooling2D(pool_size=2, padding='valid')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Conv2D(64, 3, strides=(1, 3), padding='same')(x)
x = keras.layers.LeakyReLU(alpha=0.1)(x)
x = keras.layers.MaxPooling2D(pool_size=2, padding='valid')(x)
x = keras.layers.Dropout(0.5)(x)
hidden1 = keras.layers.Flatten()(x)

hidden1 = keras.layers.Dense(100)(hidden1)
hidden1 = keras.layers.LeakyReLU(alpha=0.1)(hidden1)
hidden1 = keras.layers.Dropout(0.5)(hidden1)

hidden2 = keras.layers.Dense(200)(hidden1)
hidden2 = keras.layers.LeakyReLU(alpha=0.1)(hidden2)
hidden2 = keras.layers.Dropout(0.5)(hidden2)

hidden3 = keras.layers.Dense(100)(hidden2)
hidden3 = keras.layers.LeakyReLU(alpha=0.3)(hidden3)
hidden3 = keras.layers.Dropout(0.5)(hidden3)

output = keras.layers.Dense(4, activation='softmax')(hidden3)

In [ ]:
# Define callback - if weights don't change after 10 epochs stop training
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Create net for training
net= keras.Model(input, output, name="CNN")
net.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['sparse_categorical_accuracy'])
K.set_value(net.optimizer.learning_rate, 0.001)


## Model 2

In [ ]:
net = tf.keras.models.load_model('my_model2.keras')

In [ ]:
net.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
K.set_value(net.optimizer.learning_rate, 0.001)

In [ ]:
history = net.fit(train_ds,  epochs=100, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10), validation_data=val_ds, shuffle=True)

In [ ]:
print(np.median(history.history['sparse_categorical_accuracy']), np.median(history.history['val_sparse_categorical_accuracy']))

In [ ]:
print("Evaluate on test data")
results = net.evaluate(test_ds)
print("test loss, test acc:", results)

In [ ]:
predictions = net.predict(test_ds)
predicted_labels = tf.argmax(predictions, axis=1)

true_labels = []
for _, labels in test_ds:
    true_labels.extend(labels.numpy())

accuracy = accuracy_score(true_labels, predicted_labels)
cm = confusion_matrix(true_labels, predicted_labels)
print("Prediction accuracy: "+str(accuracy))

sns.heatmap(cm, annot=True, fmt='d', cmap='viridis', xticklabels=[0, 1, 2], yticklabels=[0, 1, 2])
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()

In [ ]:
net.save('my_model2.keras')

In [ ]:
net.summary()

## Model 1

In [ ]:
new_model = tf.keras.models.load_model('my_model.keras')

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
K.set_value(new_model.optimizer.learning_rate, 0.001)

Nastavimo trenirati taj model. Ponovo ćemo spremiti model jer će se težine u mreži promijeniti.

In [ ]:
history = new_model.fit(train_ds,  epochs=20, callbacks=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10), validation_data=val_ds, shuffle=True)

In [ ]:
print(np.median(history.history['sparse_categorical_accuracy']), np.median(history.history['val_sparse_categorical_accuracy']))


In [ ]:
print("Evaluate on test data")
results = new_model.evaluate(test_ds)
print("test loss, test acc:", results)

In [ ]:
predictions = new_model.predict(test_ds)
predicted_labels = tf.argmax(predictions, axis=1)
true_labels = []
for _, labels in test_ds:
    true_labels.extend(labels.numpy())
accuracy = accuracy_score(true_labels, predicted_labels)
print("Prediction accuracy: ", accuracy)

In [ ]:
new_model.save('my_model.keras')